### Data Overview
Load and explore the dataset with user weekly subscription transactions.  
Check the structure of the data, column types, and basic descriptive statistics to understand the available information.

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("/kaggle/input/user-purchases/user_purchases.csv")  

In [4]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   original_transaction_id  5000 non-null   int64 
 1   original_purchase_date   5000 non-null   object
 2   transaction_id           5000 non-null   int64 
 3   purchase_date            5000 non-null   object
dtypes: int64(2), object(2)
memory usage: 156.4+ KB


In [5]:
df['original_transaction_id'] = df['original_transaction_id'].astype(str)
df['transaction_id'] = df['transaction_id'].astype(str)

df['purchase_date'] = pd.to_datetime(df['purchase_date'])
df['original_purchase_date'] = pd.to_datetime(df['original_purchase_date'])



In [6]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   original_transaction_id  5000 non-null   object        
 1   original_purchase_date   5000 non-null   datetime64[ns]
 2   transaction_id           5000 non-null   object        
 3   purchase_date            5000 non-null   datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 156.4+ KB


### Conversion Rate Calculation
Calculate the conversion rate from trial to paid subscription.  
Group users by `original_transaction_id` and identify which users made at least one paid transaction after the initial trial week.

In [7]:
# total number of unique users
total_users = df['original_transaction_id'].nunique()

# users who made at least one paid transaction after the trial
converted_users = df[df['purchase_date'] > df['original_purchase_date']]['original_transaction_id'].nunique()

conversion_rate = converted_users / total_users

print("Total users:", total_users)
print("Converted users (trial -> paid):", converted_users)
print("Conversion rate:", round(conversion_rate * 100, 2), "%")


Total users: 3129
Converted users (trial -> paid): 1025
Conversion rate: 32.76 %



### ARPU Analysis
Calculate the average revenue per user (ARPU) on day 14 and forecast ARPU on day 90 of user lifetime.  
Sum all user payments within 14 and 90 days of the first transaction and divide by the total number of users.  
Use the observed revenue trend to estimate the expected user revenue by day 90.

In [8]:
import numpy as np
import pandas as pd

PRICE = 4.99

# Days from start and week number
df['days_from_start'] = (df['purchase_date'] - df['original_purchase_date']).dt.days
df['week_number'] = df['days_from_start'] // 7

# Cohort size
cohort_size = df['original_transaction_id'].nunique()

# --- Step 1: Weekly retention (paid at least once in week w, excluding trial week 0) ---
paid_weeks = (
    df[df['week_number'] >= 1][['original_transaction_id', 'week_number']]
    .drop_duplicates()
)

weekly = (
    paid_weeks.groupby('week_number')['original_transaction_id']
    .nunique()
    .rename('paid_users')
    .reset_index()
)
weekly['p_w'] = weekly['paid_users'] / cohort_size

# --- Step 2: Fit exponential decay to observed p_w ---
obs = weekly[weekly['p_w'] > 0]
x = obs['week_number'].values.astype(float)
y = obs['p_w'].values.astype(float)

if len(x) >= 2:
    coeffs = np.polyfit(x, np.log(y), deg=1)
    b = -coeffs[0]
    a = np.exp(coeffs[1])
    exp_decay = lambda w: a * np.exp(-b * w)
else:
    exp_decay = lambda w: y.mean() if len(y) > 0 else 0.0

# --- Step 3: Build forecast weeks up to 12 (≈90 days) ---
weeks = np.arange(1, 13)
p_w_used = []
for w in weeks:
    actual = weekly.loc[weekly['week_number'] == w, 'p_w']
    if not actual.empty:
        p_w_used.append(actual.values[0])
    else:
        p_w_used.append(exp_decay(w))

# --- Step 4: Compute ARPU ---
arpu_d14 = sum([p * PRICE for w, p in zip(weeks, p_w_used) if w <= 2])
arpu_d90 = sum([p * PRICE for w, p in zip(weeks, p_w_used)])

print("Cohort size (trial starters):", cohort_size)
print("ARPU Day 14: $", round(arpu_d14, 2))
print("ARPU Day 90 (forecast): $", round(arpu_d90, 2))


Cohort size (trial starters): 3129
ARPU Day 14: $ 2.33
ARPU Day 90 (forecast): $ 3.01
